In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

sc = SparkSession.builder.appName("word-count").master("local").getOrCreate()
sc
sc.conf.set("spark.sql.shuffle.partitions", 5)

In [2]:
df = sc.readStream.format("socket").option("host", "localhost").option("port", "9999").load()
df.printSchema()

root
 |-- value: string (nullable = true)



In [3]:
split_df = df.withColumn("words", split(col("value"), " "))

In [4]:
explode_df = split_df.withColumn("explode", explode(col("words"))).drop("value", "words")

In [ ]:
res_df = explode_df.groupBy("explode").count()
res_df.writeStream.format("console").outputMode("complete").start().awaitTermination()
